adding dependencies


In [6]:
import sys, os
import pandas as pd
import numpy as np


importing modules

In [7]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [8]:
tf.__version__

'2.2.0'

importing the data

In [9]:
df=pd.read_csv('fer2013.csv')

In [10]:
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [12]:
print(df["Usage"].value_counts())


Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64


emotions is divided into (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral) categories<br>
the pixels contains 48*48 coloums of data <br>
usage is basically the use of the data as training, public test or  private test<br>

In [13]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


declaring the test and train variables
here we declare empty list for the variables

In [14]:
X_train,y_train,X_test,y_test=[],[],[],[]

As the pixel coloumn contains 48 values so we have convert into list <br>
We to it by iterating it through a for loop

In [15]:
for index,row in df.iterrows():
    val=row["pixels"].split(" ")
    try:
        if "Training" in row['Usage']: #-------if value of a usage is training then we apppend the pixels in X_train
            X_train.append(np.array(val,'float32')) # as we have already stored pixels in list "val" so we appendand convert it into float 32
            y_train.append(row['emotion'])
        elif "PublicTest" in row['Usage']: # if usage is given as public test then we append the data in test part of the dataset
            X_test.append(np.array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [16]:
#print(val)

In [17]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


As keras only accepts the np array format input so we have to convert all the train and test split into np array 

In [18]:
X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

In [19]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


normalizing the data bw 0 and 1 for better predictions

step-1) Substaracting mean<br>
step2) dividing by stanadrd Deviation

In [17]:
#X_train-=np.mean(X_train,axis=0)
#X_train/=np.std(X_train,axis=0)

#X_test-=np.mean(X_test,axis=0)
#X_test/=np.std(X_test,axis=0)

In [20]:
X_train.min(),X_train.max()

(0.0, 255.0)

In [21]:
X_test.min(),X_test.max()

(0.0, 255.0)

### Min max scaling

### Xnorm=$\frac{x-minvalue}{mavalue-minvalue}$

In [22]:
X_train=X_train/X_train.max()

In [23]:
X_test=X_test/X_test.max()

In [24]:
print(X_train)

[[0.27450982 0.3137255  0.32156864 ... 0.41568628 0.42745098 0.32156864]
 [0.5921569  0.5882353  0.5764706  ... 0.75686276 0.7176471  0.72156864]
 [0.90588236 0.83137256 0.6117647  ... 0.34509805 0.43137255 0.59607846]
 ...
 [0.2901961  0.31764707 0.34117648 ... 0.7372549  0.73333335 0.73333335]
 [0.87058824 0.8901961  0.79607844 ... 0.53333336 0.53333336 0.5254902 ]
 [0.7647059  0.78039217 0.8039216  ... 0.02352941 0.05882353 0.14901961]]


In [25]:
print(X_test)

[[0.99607843 0.99607843 0.99607843 ... 0.16470589 0.5058824  0.7058824 ]
 [0.6117647  0.72156864 0.7764706  ... 0.6745098  0.654902   0.6313726 ]
 [0.27058825 0.4627451  0.23921569 ... 0.34509805 0.34117648 0.3529412 ]
 ...
 [1.         1.         1.         ... 0.1882353  0.19607843 0.18039216]
 [0.12941177 0.09803922 0.12156863 ... 0.01568628 0.01960784 0.01568628]
 [0.23921569 0.24705882 0.23137255 ... 0.44313726 0.64705884 0.7058824 ]]


In [26]:
print(y_train)

[0. 0. 2. ... 4. 0. 4.]


decralaring some values

In [27]:
num_features = 64 # no.of features of conv2d layer
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48 #pixel 


In [28]:
X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)

In [29]:
print(X_test)

[[[[0.99607843]
   [0.99607843]
   [0.99607843]
   ...
   [0.0627451 ]
   [0.        ]
   [0.6313726 ]]

  [[0.99607843]
   [0.99607843]
   [0.99607843]
   ...
   [0.06666667]
   [0.        ]
   [0.47843137]]

  [[0.99607843]
   [0.99607843]
   [0.99607843]
   ...
   [0.10196079]
   [0.        ]
   [0.44705883]]

  ...

  [[0.25882354]
   [0.3372549 ]
   [0.3882353 ]
   ...
   [1.        ]
   [0.9882353 ]
   [0.99607843]]

  [[0.32941177]
   [0.36078432]
   [0.36078432]
   ...
   [0.9254902 ]
   [1.        ]
   [1.        ]]

  [[0.3647059 ]
   [0.3529412 ]
   [0.33333334]
   ...
   [0.16470589]
   [0.5058824 ]
   [0.7058824 ]]]


 [[[0.6117647 ]
   [0.72156864]
   [0.7764706 ]
   ...
   [0.6156863 ]
   [0.6039216 ]
   [0.5882353 ]]

  [[0.57254905]
   [0.7137255 ]
   [0.78039217]
   ...
   [0.6313726 ]
   [0.6039216 ]
   [0.5882353 ]]

  [[0.5294118 ]
   [0.6901961 ]
   [0.7647059 ]
   ...
   [0.64705884]
   [0.6313726 ]
   [0.60784316]]

  ...

  [[0.10980392]
   [0.09803922]
   [0.0

In [30]:
y_train=np_utils.to_categorical(y_train, num_classes=num_labels)
y_test=np_utils.to_categorical(y_test, num_classes=num_labels)

desinging the cnn

In [31]:
#1st Layer

In [32]:
model=Sequential()

In [33]:
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu",input_shape=(X_train.shape[1:])))#kernel size is the filter size that we are going to superimpose n images,relu is recifier activation function 
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) # mqx pool function choosesthe max value from the pool and ouput is fomed with that
model.add(Dropout(0.5)) #drop out somethings so that model doesnt overfit

In [34]:
#2nd layer

In [35]:
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))#kernel size is the filter size that we are going to superimpose n images,relu is recifier activation function 
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) # mqx pool function choosesthe max value from the pool and ouput is fomed with that
model.add(Dropout(0.5)) #drop out somethings so that model doesnt overfit

3rd layer- in this layer we double the filters

In [36]:
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) 

<img src="capture.png" width="400"/>

TO flatten the function

In [37]:
model.add(Flatten()) # reshapes the function to the number of elements contained

In [38]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

In [39]:
model.add(Dense(num_labels, activation='softmax')) # softmax as we have to choose bw the available labels given

In [40]:
#compiling the model

In [41]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=["Accuracy"])

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0

In [43]:
#Training the model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True)




Epoch 1/30
449/449 [==============================] - 247s 551ms/step - loss: 1.8170 - accuracy: 0.0000e+00 - val_loss: 1.8171 - val_accuracy: 0.0000e+00
Epoch 2/30
449/449 [==============================] - 251s 560ms/step - loss: 1.8118 - accuracy: 0.0000e+00 - val_loss: 1.8170 - val_accuracy: 0.0000e+00
Epoch 3/30
449/449 [==============================] - 239s 532ms/step - loss: 1.7371 - accuracy: 0.0000e+00 - val_loss: 1.6954 - val_accuracy: 0.0000e+00
Epoch 4/30
449/449 [==============================] - 240s 535ms/step - loss: 1.6340 - accuracy: 0.0000e+00 - val_loss: 1.5448 - val_accuracy: 0.0000e+00
Epoch 5/30
449/449 [==============================] - 281s 625ms/step - loss: 1.5629 - accuracy: 0.0000e+00 - val_loss: 1.4920 - val_accuracy: 0.0000e+00
Epoch 6/30
449/449 [==============================] - 291s 648ms/step - loss: 1.4913 - accuracy: 0.0000e+00 - val_loss: 1.3957 - val_accuracy: 0.0000e+00
Epoch 7/30
449/449 [==============================] - 309s 688ms/step - loss

In [44]:
fer_json = model.to_json()
with open("fer_minmax.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer_minmax.h5")